# Preprocessing Ames Housing Data

## Training Data

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Obtain Train Data
filename = '../raw/train.csv'
dataset = pd.read_csv(filename, keep_default_na=True)

In [3]:
# Check dataset shape
observations, variables = dataset.shape

In [4]:
# Check for missing values
missing_value_counts = dataset.isnull().sum()
df_missing = pd.DataFrame(data=[missing_value_counts.index, missing_value_counts.values]).T
df_missing = df_missing.rename(columns={0:'column', 1:'n_missing'})

In [5]:
# Identify worst offenders (missing over 75% of values)
worst_offenders_train = df_missing[df_missing['n_missing'] > observations * .25]
worst_offenders_train

,column,n_missing
6,Alley,1369
57,FireplaceQu,690
72,PoolQC,1453
73,Fence,1179
74,MiscFeature,1406


In [6]:
# Remove worst offenders
dataset = dataset.drop(columns = list(worst_offenders_train.column))
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non

In [7]:
# Check remaining NA'S columns
missing_value_counts = dataset.isnull().sum()
df_missing = pd.DataFrame(data=[missing_value_counts.index, missing_value_counts.values]).T
df_missing = df_missing.rename(columns={0:'column', 1:'n_missing'})
df_missing_sort = df_missing.sort_values(['n_missing'], ascending=0)
df_missing_sort[df_missing_sort.n_missing>0]

,column,n_missing
3,LotFrontage,259
56,GarageType,81
62,GarageCond,81
57,GarageYrBlt,81
58,GarageFinish,81
61,GarageQual,81
34,BsmtFinType2,38
31,BsmtExposure,38
32,BsmtFinType1,37
30,BsmtCond,37


In [8]:
# Check for rows with missing values
missing_value_counts = dataset.isnull().sum(axis=1)
df_missing = pd.DataFrame(data=[missing_value_counts.index, missing_value_counts.values]).T
df_missing = df_missing.rename(columns={0:'row', 1:'n_missing'})
print(df_missing[df_missing['n_missing']>0].shape[0])

366


In [9]:
# Impute Remaining NaN with mean: median or .mode
dataset = dataset.replace({"NA": np.NaN})
dataset['LotFrontage'].replace({np.NaN: dataset['LotFrontage'].mean()}, inplace=True)
dataset['GarageType'].replace({np.NaN: dataset['GarageType'].mode()[0]}, inplace=True)
dataset['GarageCond'].replace({np.NaN: dataset['GarageCond'].mode()[0]}, inplace=True)
dataset['GarageYrBlt'].replace({np.NaN: dataset['GarageYrBlt'].mean()}, inplace=True)
dataset['GarageFinish'].replace({np.NaN: dataset['GarageFinish'].mode()[0]}, inplace=True)
dataset['GarageQual'].replace({np.NaN: dataset['GarageQual'].mode()[0]}, inplace=True)
dataset['BsmtFinType2'].replace({np.NaN: dataset['BsmtFinType2'].mode()[0]}, inplace=True)
dataset['BsmtExposure'].replace({np.NaN: dataset['BsmtExposure'].mode()[0]}, inplace=True)
dataset['BsmtFinType1'].replace({np.NaN: dataset['BsmtFinType1'].mode()[0]}, inplace=True)
dataset['BsmtCond'].replace({np.NaN: dataset['BsmtCond'].mode()[0]}, inplace=True)
dataset['BsmtQual'].replace({np.NaN: dataset['BsmtQual'].mode()[0]}, inplace=True)
dataset['MasVnrArea'].replace({np.NaN: dataset['MasVnrArea'].mode()[0]}, inplace=True)
dataset['MasVnrType'].replace({np.NaN: dataset['MasVnrType'].mode()[0]}, inplace=True)
dataset['Electrical'].replace({np.NaN: dataset['Electrical'].mode()[0]}, inplace=True)
# Confirm no more missing values
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non

In [10]:
# Save processed  training file
filename = '../interim/train.csv'
dataset.to_csv(filename)

## Test Data

In [11]:
# Obtain Test Data
filename = '../raw/test.csv'
dataset = pd.read_csv(filename, keep_default_na=True)

In [12]:
# Identify worst offenders (missing over 75% of values)
worst_offenders = df_missing[df_missing['n_missing'] > observations * .25]
worst_offenders

,row,n_missing


In [13]:
# Remove worst offenders removed from training set.
dataset = dataset.drop(columns = list(worst_offenders_train.column))
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 75 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-null object
Exterior2nd      1458 non

In [14]:
# Check remaining NA'S columns
missing_value_counts = dataset.isnull().sum()
df_missing = pd.DataFrame(data=[missing_value_counts.index, missing_value_counts.values]).T
df_missing = df_missing.rename(columns={0:'column', 1:'n_missing'})
df_missing_sort = df_missing.sort_values(['n_missing'], ascending=0)
df_missing_sort[df_missing_sort.n_missing>0]

,column,n_missing
3,LotFrontage,227
57,GarageYrBlt,78
58,GarageFinish,78
61,GarageQual,78
62,GarageCond,78
56,GarageType,76
30,BsmtCond,45
31,BsmtExposure,44
29,BsmtQual,44
32,BsmtFinType1,42


In [15]:
# Impute Remaining NaN with mean: median or .mode
dataset = dataset.replace({"NA": np.NaN})
dataset['LotFrontage'].replace({np.NaN: dataset['LotFrontage'].mean()}, inplace=True)
dataset['GarageType'].replace({np.NaN: dataset['GarageType'].mode()[0]}, inplace=True)
dataset['GarageCond'].replace({np.NaN: dataset['GarageCond'].mode()[0]}, inplace=True)
dataset['GarageYrBlt'].replace({np.NaN: dataset['GarageYrBlt'].mean()}, inplace=True)
dataset['GarageFinish'].replace({np.NaN: dataset['GarageFinish'].mode()[0]}, inplace=True)
dataset['GarageQual'].replace({np.NaN: dataset['GarageQual'].mode()[0]}, inplace=True)
dataset['BsmtFinType2'].replace({np.NaN: dataset['BsmtFinType2'].mode()[0]}, inplace=True)
dataset['BsmtExposure'].replace({np.NaN: dataset['BsmtExposure'].mode()[0]}, inplace=True)
dataset['BsmtFinType1'].replace({np.NaN: dataset['BsmtFinType1'].mode()[0]}, inplace=True)
dataset['BsmtCond'].replace({np.NaN: dataset['BsmtCond'].mode()[0]}, inplace=True)
dataset['BsmtQual'].replace({np.NaN: dataset['BsmtQual'].mode()[0]}, inplace=True)
dataset['MasVnrArea'].replace({np.NaN: dataset['MasVnrArea'].mode()[0]}, inplace=True)
dataset['MasVnrType'].replace({np.NaN: dataset['MasVnrType'].mode()[0]}, inplace=True)
dataset['Electrical'].replace({np.NaN: dataset['Electrical'].mode()[0]}, inplace=True)
dataset['MSZoning'].replace({np.NaN: dataset['MSZoning'].mode()[0]}, inplace=True)
dataset['Utilities'].replace({np.NaN: dataset['Utilities'].mode()[0]}, inplace=True)
dataset['Functional'].replace({np.NaN: dataset['Functional'].mode()[0]}, inplace=True)
dataset['BsmtHalfBath'].replace({np.NaN: dataset['BsmtHalfBath'].mode()[0]}, inplace=True)
dataset['BsmtFullBath'].replace({np.NaN: dataset['BsmtFullBath'].mode()[0]}, inplace=True)
dataset['Exterior2nd'].replace({np.NaN: dataset['Exterior2nd'].mode()[0]}, inplace=True)
dataset['BsmtFinSF1'].replace({np.NaN: dataset['BsmtFinSF1'].mean()}, inplace=True)
dataset['BsmtFinSF2'].replace({np.NaN: dataset['BsmtFinSF2'].mean()}, inplace=True)
dataset['BsmtUnfSF'].replace({np.NaN: dataset['BsmtUnfSF'].mean()}, inplace=True)
dataset['KitchenQual'].replace({np.NaN: dataset['KitchenQual'].mode()[0]}, inplace=True)
dataset['TotalBsmtSF'].replace({np.NaN: dataset['TotalBsmtSF'].mean()}, inplace=True)
dataset['Exterior1st'].replace({np.NaN: dataset['Exterior1st'].mode()[0]}, inplace=True)
dataset['GarageArea'].replace({np.NaN: dataset['GarageArea'].mean()}, inplace=True)
dataset['SaleType'].replace({np.NaN: dataset['SaleType'].mode()[0]}, inplace=True)
dataset['GarageCars'].replace({np.NaN: dataset['GarageCars'].mode()[0]}, inplace=True)
# Confirm no more missing values
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 75 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1459 non-null object
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1459 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1459 non-null object
Exterior2nd      1459 non

In [16]:
# Save processed test file
filename = '../interim/test.csv'
dataset.to_csv(filename)